<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/SeriesdeTiempo/EMAE_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import json

import requests
from requests.auth import HTTPBasicAuth

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/alphacast.json", 'r') as f:
    api_token= json.load(f)

Mounted at /content/drive


In [3]:
# Definir user key para la API
user_key = api_token["key"]

In [4]:
# EMAE
emae = requests.get('https://api.alphacast.io/datasets/5331/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('emae.csv','wb').write(emae.content)
emae = pd.read_csv('emae.csv',delimiter = ',')

In [5]:
emae.set_index('Date', inplace = True)
emae = emae.loc['2016-02-01':]
y=emae['Emae - current_prices_mom']

In [43]:
# IPI
ipi = requests.get('https://api.alphacast.io/datasets/5679/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('ipi.csv','wb').write(ipi.content)
ipi = pd.read_csv('ipi.csv',delimiter = ',')


ipi['Date'] = pd.to_datetime(ipi['Date'])
ipi.set_index('Date', inplace = True)
ipi = ipi[(ipi['Division'] == "IPI Manufacturero") & (ipi['Seasonal Adjustment'] == "Seasonally Adjusted")]
ipi = ipi['Value - sa_MoM']
ipi.columns = ['Value - sa_MoM']
ipi = ipi.loc['2016-02-01':]

In [44]:
# ISAC
isac = requests.get('https://api.alphacast.io/datasets/5565/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('isac.csv','wb').write(isac.content)
isac = pd.read_csv('isac.csv',delimiter = ',')


isac['Date'] = pd.to_datetime(isac['Date'])
isac.set_index('Date', inplace = True)
isac = isac['Nivel general - sa_MoM']
isac.columns = ['Nivel general - sa_MoM']
isac = isac.loc['2016-02-01':]

In [45]:
isac=pd.Series(isac, name='ISAC')
ipi=pd.Series(ipi, name='IPI')

In [46]:
X = pd.concat([ipi, isac], axis=1)

In [55]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X[:-1], y[:-1])
Yest = lr.predict(X[:-1])

In [57]:
print('DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE')
print()
print('Valor de la pendiente o coeficiente "a":')
print(lr.coef_)
print('Valor de la intersección o coeficiente "b":')
print(lr.intercept_)
print()
print('La ecuación del modelo es igual a:')
print('y = ', lr.coef_, 'X + ', lr.intercept_)
print()
print('Precisión del modelo:')
print(lr.score(X[:-1], y[:-1]))

DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE

Valor de la pendiente o coeficiente "a":
[-0.0344089   0.15729643]
Valor de la intersección o coeficiente "b":
0.1522808823148907

La ecuación del modelo es igual a:
y =  [-0.0344089   0.15729643] X +  0.1522808823148907

Precisión del modelo:
0.15280636664497793


In [58]:
lr.coef_[0]

-0.034408903516004535

In [59]:
emae_predict=X[-1:]['IPI']*lr.coef_[0]+X[-1:]['ISAC']*lr.coef_[1]+lr.intercept_

In [60]:
emae_predict

Date
2023-04-01   -0.288719
dtype: float64